In [1]:
import torch  
import torch.nn as nn
import numpy as np
import math
from PIL import Image
import cv2

ModuleNotFoundError: No module named 'torch'

In [2]:
!pip install torch

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Dicks\anaconda3\envs\open3d\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\Dicks\anaconda3\envs\open3d\lib\site-packages\pip\_vendor\urllib3\response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Users\Dicks\anaconda3\envs\open3d\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "C:\Users\Dicks\anaconda3\envs\open3d\lib\http\client.py", line 459, in read
    n = self.readinto(b)
  File "C:\Users\Dicks\anaconda3\envs\open3d\lib\http\client.py", line 503, in readinto
    n = self.fp.readinto(b)
  File "C:\Users\Dicks\anaconda3\envs\open3d\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\Dicks\anaconda3\envs\open3d\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Users\Dicks\ana

In [ ]:
def SSIM(x, y, C1=1e-4, C2=9e-4, kernel_size=3, stride=1):
    """
    Structural SIMilarity (SSIM) distance between two images.

    Parameters
    ----------
    x,y : torch.Tensor [B,3,H,W]
        Input images
    C1,C2 : float
        SSIM parameters
    kernel_size,stride : int
        Convolutional parameters

    Returns
    -------
    ssim : torch.Tensor [1]
        SSIM distance
    """
    pool2d = nn.AvgPool2d(kernel_size, stride=stride)
    refl = nn.ReflectionPad2d(1)

    x, y = refl(x), refl(y)
    mu_x = pool2d(x)
    mu_y = pool2d(y)

    mu_x_mu_y = mu_x * mu_y
    mu_x_sq = mu_x.pow(2)
    mu_y_sq = mu_y.pow(2)

    sigma_x = pool2d(x.pow(2)) - mu_x_sq
    sigma_y = pool2d(y.pow(2)) - mu_y_sq
    sigma_xy = pool2d(x * y) - mu_x_mu_y
    v1 = 2 * sigma_xy + C2
    v2 = sigma_x + sigma_y + C2

    # Where is the global ssim
    # I think AvgPool basically calculate the LOCAL SSIM
    # While Global SSIM is computer from the sigma's and the mu's. But we should test it out.
    
    # SSIM Nominator and denominator
    # SSIM_N = (2*mu_xy + C)* (2*sigmaxy+C2)
    # SSIM_D = (mu_x^2 + mu_y^2+C1) * (sigma_x^2 + sigma_y^2 + C2)
    ssim_n = (2 * mu_x_mu_y + C1) * v1
    ssim_d = (mu_x_sq + mu_y_sq + C1) * v2
    ssim = ssim_n / ssim_d

    return ssim

In [ ]:
def ssim_to_img(ssim):
    arr = np.asarray(ssim[0])
    arr = arr.transpose((1,2,0))
    return arr

In [ ]:
tensorify = lambda x: torch.Tensor(x.transpose((2, 0, 1))).unsqueeze(0).float().div(255.0)

In [ ]:
left = tensorify(cv2.imread("images/left.jpg"))
right = tensorify(cv2.imread("images/right.jpg"))

similarity = SSIM(left,right)
ssim_img = ssim_to_img(similarity)
#import matplotlib.pyplot as plt
#plt.plot(ssim_img)
#imshow(ssim_img)
p